In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import tensorflow as tf
import numpy as np

In [2]:
# Load student and teacher data from CSV files
students_df = pd.read_csv('students.csv')
teachers_df = pd.read_csv('teachers.csv')


In [3]:
# Combine interests and CGPA into a single feature vector for clustering
student_features = students_df[['CGPA']].values

In [4]:
# Define the number of clusters (teachers)
num_clusters = len(teachers_df)

In [5]:
# Create a KMeans clustering model
kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=0).fit(student_features)

In [6]:
# Assign each student to a cluster (teacher)
students_df['Cluster'] = kmeans.labels_

In [7]:
# Initialize teacher assignments
teacher_assignments = {teacher: [] for teacher in teachers_df['Teacher_name']}

In [8]:
# Iterate through students and assign them to the nearest matching teacher
for _, student in students_df.iterrows():
    cluster_label = student['Cluster']
    teacher_name = teachers_df.iloc[cluster_label]['Teacher_name']
    student_name = student['Student_name']
    cgpa = student['CGPA']
    teacher_assignments[teacher_name].append((student_name, cgpa))

In [9]:
# Print the assignments with teacher name, student name, and CGPA
for teacher, students in teacher_assignments.items():
    print(f"Teacher: {teacher}")
    for student, cgpa in students:
        print(f"  Student: {student}, CGPA: {cgpa}")

Teacher: Pankaj
  Student: Lucas Cooper, CGPA: 6.3
  Student: Charlotte Hayes, CGPA: 6.2
  Student: Caleb Smith, CGPA: 6.3
  Student: Daniel Robinson, CGPA: 7.8
  Student: Chloe Moore, CGPA: 7.6
Teacher: Amit
  Student: Ethan Turner, CGPA: 8.4
  Student: Sophia Jenkins, CGPA: 8.2
  Student: Mia Nelson, CGPA: 6.5
  Student: Oliver Rivera, CGPA: 8.2
  Student: Harper Gray, CGPA: 8.3
  Student: Henry Martinez, CGPA: 6.7
Teacher: Akshay
  Student: Olivia Bennett, CGPA: 9.4
  Student: Emma Parker, CGPA: 9.5
  Student: Noah Harrison, CGPA: 9.6
  Student: Amelia Reed, CGPA: 8.8
  Student: Elijah Brooks, CGPA: 9.4
  Student: Benjamin Wright, CGPA: 8.6
  Student: Scarlett Adams, CGPA: 9.4
Teacher: Sachin
  Student: Ava Mitchell, CGPA: 8.0
  Student: Mason Foster, CGPA: 8.1
  Student: Jackson Walker, CGPA: 7.2
  Student: Abigail Simmons, CGPA: 7.3
  Student: Evelyn Taylor, CGPA: 7.2
  Student: Samuel Davis, CGPA: 7.9
  Student: Lily Johnson, CGPA: 8.1
Teacher: Ranjodh
  Student: Liam Anderson, C

In [10]:
# Convert K-Means model to TFLite
kmeans_model = kmeans
converter = tf.lite.TFLiteConverter.from_keras_model(kmeans_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('kmeans_model.tflite', 'wb') as f:
    f.write(tflite_model)

AttributeError: 'KMeans' object has no attribute 'call'

In [11]:
import joblib

# Save the K-Means model
joblib.dump(kmeans, 'kmeans_model.joblib')

# Load the K-Means model when needed
kmeans = joblib.load('kmeans_model.joblib')
